**Imports**

In [78]:
import h5py
#
from sklearn.model_selection import train_test_split

#Evaluaiton du modèle
from sklearn.metrics import classification_report

import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
from torch.utils.data import Subset
import torch.nn as nn
import torch.nn.functional as F

#Scheduler pour le learning rate
from torch.optim.lr_scheduler import StepLR

#Entrainement du modèle
import torch.optim as optim

#Visualisation des résultats
from torch.utils.tensorboard import SummaryWriter

#Fonction de perte
from sklearn.utils.class_weight import compute_class_weight


Chargement des datasets

In [79]:
dataset_augmented_path = 'D:/ApprentissageMachineTPFinal/data/dog_dataset_aug_normal.h5'

**Classe pour charger le dataset aug**

In [80]:
# Classe personnalisée pour charger les données
class DogBreedDataset(Dataset):
    def __init__(self, dataset_augmented_path, transform=None):
        self.file_path = dataset_augmented_path
        self.transform = transform
        
        # Charger uniquement les dimensions
        with h5py.File(self.file_path, "r") as f:
            if "images" in f:
                self.data_len = f["images"].shape[0] # type: ignore
            else:
                raise KeyError("Dataset 'images' not found in the HDF5 file.")
        
    def __len__(self):
        return self.data_len

    def __getitem__(self, idx):
        with h5py.File(self.file_path, "r") as f:
            image = f["images"][idx] / 255.0  # type: ignore # Normalisation
            label = f["labels"][idx] # type: ignore
        
        # Appliquer une transformation éventuelle
        if self.transform:
            image = self.transform(image)

        # Format PyTorch
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label

# Initialisation des datasets
dataset = DogBreedDataset(dataset_augmented_path)

# Division en ensembles
indices = list(range(len(dataset)))
train_indices, temp_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

# Création des DataLoaders
train_loader = DataLoader(Subset(dataset, train_indices), batch_size=64, shuffle=True)
val_loader = DataLoader(Subset(dataset, val_indices), batch_size=64, shuffle=False)
test_loader = DataLoader(Subset(dataset, test_indices), batch_size=64, shuffle=False)

**Creation du modele**

In [81]:
class DogBreedCNN(nn.Module):
    def __init__(self, num_classes=120):
        super(DogBreedCNN, self).__init__()

        # Convolutional layers with BatchNorm and ReLU
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # Reduce spatial dimensions by half
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, num_classes)

        self.dropout = nn.Dropout(0.5)


    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)

        # Flatten the output
        x = x.view(x.size(0), -1)
        #print(f"Shape after flattening: {x.shape}")

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        
        # Fully connected layers
        x = self.fc2(x)

        return x

model = DogBreedCNN(num_classes=120)

Test manuel d'etape

In [82]:
#images, labels = next(iter(train_loader))
#outputs = model(images)
#print(f"Model outputs shape: {outputs.shape}")

Vérification le contenu des données dans le DataLoader

In [83]:
for images, labels in train_loader:
    print(f"Images shape: {images.shape}")
    print(f"Sample images shape: {images.shape}")
    print(f"Sample labels: {labels}")
    break

Images shape: torch.Size([64, 3, 224, 224])
Sample images shape: torch.Size([64, 3, 224, 224])
Sample labels: tensor([103,  65,  10,  73,  63,  93, 109, 109, 107,  30,  78,  67,   5,  63,
        117,  17,  62, 101,  52,   5,  50,  74,  77,   6,  49,  37,  12,  48,
         63, 106,  31, 119,   2,  42,  67,  60,  46,  69, 100,  86,  17,  31,
         63,  42, 105,  23,  77, 114, 102,  30, 117, 111,  14,   4,   0, 106,
          7,  71,   7,  91,  59,  72,  78,  88])


**Entrainement aug**

In [84]:
# Configurations
learning_rate = 0.0001
num_epochs = 30

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
    scheduler.step()


Epoch [1/30], Loss: 4.7771


d:\ApprentissageMachineTPFinal\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch [2/30], Loss: 4.6818
Epoch [3/30], Loss: 4.6025
Epoch [4/30], Loss: 4.5374
Epoch [5/30], Loss: 4.5050
Epoch [6/30], Loss: 4.4617
Epoch [7/30], Loss: 4.4470
Epoch [8/30], Loss: 4.4215
Epoch [9/30], Loss: 4.4051
Epoch [10/30], Loss: 4.3722
Epoch [11/30], Loss: 4.3552
Epoch [12/30], Loss: 4.3379
Epoch [13/30], Loss: 4.3158
Epoch [14/30], Loss: 4.3028
Epoch [15/30], Loss: 4.2804
Epoch [16/30], Loss: 4.2515
Epoch [17/30], Loss: 4.2206
Epoch [18/30], Loss: 4.2075
Epoch [19/30], Loss: 4.1876
Epoch [20/30], Loss: 4.1753
Epoch [21/30], Loss: 4.1554
Epoch [22/30], Loss: 4.1529
Epoch [23/30], Loss: 4.1348
Epoch [24/30], Loss: 4.1198
Epoch [25/30], Loss: 4.1119
Epoch [26/30], Loss: 4.1087
Epoch [27/30], Loss: 4.0871
Epoch [28/30], Loss: 4.0843
Epoch [29/30], Loss: 4.0644
Epoch [30/30], Loss: 4.0549


**Evaluation du modele**

In [85]:
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Rapport de classification
print(classification_report(y_true, y_pred, zero_division=1))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        62
           1       1.00      0.00      0.00        27
           2       0.03      0.02      0.02        51
           3       0.00      0.00      0.00        54
           4       1.00      0.00      0.00        56
           5       0.05      0.30      0.09        54
           6       1.00      0.00      0.00        22
           7       0.12      0.17      0.14        70
           8       0.06      0.26      0.09        47
           9       0.09      0.06      0.08        47
          10       0.00      0.00      0.00        49
          11       0.03      0.12      0.05        49
          12       1.00      0.00      0.00        20
          13       1.00      0.00      0.00        60
          14       1.00      0.00      0.00        38
          15       0.00      0.00      0.00        46
          16       0.00      0.00      0.00        51
          17       1.00    

Visualisez les Résultats

In [86]:
val_loss=0.0
writer = SummaryWriter()
for epoch in range(num_epochs):
    writer.add_scalar('Loss/train', running_loss, epoch)
    writer.add_scalar('Loss/val', val_loss, epoch)
writer.close()
